## Create CONSTANTS

In [49]:
DATASET = [
    {
        'Tid': 'T1',
        'Item': ['a', 'b', 'c', 'd'],
        'Quantity': [5,2,1,2]
    },
    {
        'Tid': 'T2',
        'Item': ['a', 'c', 'd', 'g'],
        'Quantity': [1,1,1,3]
    },
    {
        'Tid': 'T3',
        'Item': ['a', 'c', 'f'],
        'Quantity': [1,1,1]
    },
    {
        'Tid': 'T4',
        'Item': ['a', 'f', 'g'],
        'Quantity': [1,4,2]
    },
    {
        'Tid': 'T5',
        'Item': ['a', 'g'],
        'Quantity': [1,2]
    },
    {
        'Tid': 'T6',
        'Item': ['b', 'c', 'd', 'e'],
        'Quantity': [3,2,3,1]
    },
    {
        'Tid': 'T7',
        'Item': ['c', 'e'],
        'Quantity': [6,4]
    },
    {
        'Tid': 'T8',
        'Item': ['e', 'f'],
        'Quantity': [1,3]
    }
]
UNIT_UTILITY = {
    'a': 3,
    'b': 6,
    'c': -3,
    'd': 12,
    'e': -5,
    'f': -2,
    'g': -1
}

EPM = {
    'min_util': 30,
    'min_per': 1,
    'max_per': 5,
    'min_avg': 1,
    'max_avg': 3
}

### Show global variable

In [50]:
for i in DATASET:
    print(i)

{'Tid': 'T1', 'Item': ['a', 'b', 'c', 'd'], 'Quantity': [5, 2, 1, 2]}
{'Tid': 'T2', 'Item': ['a', 'c', 'd', 'g'], 'Quantity': [1, 1, 1, 3]}
{'Tid': 'T3', 'Item': ['a', 'c', 'f'], 'Quantity': [1, 1, 1]}
{'Tid': 'T4', 'Item': ['a', 'f', 'g'], 'Quantity': [1, 4, 2]}
{'Tid': 'T5', 'Item': ['a', 'g'], 'Quantity': [1, 2]}
{'Tid': 'T6', 'Item': ['b', 'c', 'd', 'e'], 'Quantity': [3, 2, 3, 1]}
{'Tid': 'T7', 'Item': ['c', 'e'], 'Quantity': [6, 4]}
{'Tid': 'T8', 'Item': ['e', 'f'], 'Quantity': [1, 3]}


In [51]:
print(UNIT_UTILITY)

{'a': 3, 'b': 6, 'c': -3, 'd': 12, 'e': -5, 'f': -2, 'g': -1}


In [52]:
print(EPM)

{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}


## Preparation Procedure

The Transaction Weight Utility (TWU) of item \( i \) is given by:

Def 6
Def 4
\[
{TWU}(i) = \sum_{{X \in T_j \\ T_j \subseteq D}} P_U(T_j)
\]

calculate maxPer(i), avgPer(i)
Def 1, Def 2

rearrange order
Def 7


In [53]:
def create_periodic_high_util_itemset(dataset: list, unit_utility: dict, epm: dict) -> list:
    # for i in dataset:
    #     print(i)
    # calculate TWU of each item in dataset
    twu_dict = dict()
    twu_transaction_dict = dict()
    ps_dict = dict()
    transaction_dict = dict()
    for i in dataset:
        twu_transaction_dict[i['Tid']] = 0
        for j in range(len(i['Item'])):
            if i['Item'][j] not in twu_dict:
                twu_dict[i['Item'][j]] = 0
                transaction_dict[i['Item'][j]] = []
                ps_dict[i['Item'][j]] = []

            twu_transaction_dict[i['Tid']] += i['Quantity'][j] * unit_utility[i['Item'][j]] if unit_utility[i['Item'][j]] > 0 else 0
            transaction_dict[i['Item'][j]].append(int(i['Tid'][1:]))

        for j in range(len(i['Item'])):
            twu_dict[i['Item'][j]] += twu_transaction_dict[i['Tid']]
    
    # print(twu_transaction_dict)
    for i in transaction_dict.keys():
        ps_dict[i].append(transaction_dict[i][0])
        for j in range(1, len(transaction_dict[i])):
            ps_dict[i].append(transaction_dict[i][j] - transaction_dict[i][j - 1])
    
    temp_dataset = list()
    eucs_dict = dict()

    for i in dataset:
        temp_item_quantity = list()
        for j in range(len(i['Item'])):
            avg_item = sum(ps_dict[i['Item'][j]])/len(ps_dict[i['Item'][j]])
            # print((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
            if twu_dict[i['Item'][j]] > epm['min_util'] and max(ps_dict[i['Item'][j]]) <= epm['max_per']\
            and avg_item >= epm['min_avg']:
                
                temp_item_quantity.append((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))

        temp_item_quantity.sort(key=lambda x: x[3])
        print(temp_item_quantity)
        temp_item_quantity.sort(key=lambda x: x[2])
        print(temp_item_quantity)
        # print(i)
        temp_dataset.append(
            {
                'Tid': i['Tid'],
                'Item': list(map(lambda x: x[0], temp_item_quantity)),
                'Quantity': list(map(lambda x: x[1], temp_item_quantity))
            }
        )
        # print(i, end='\n\n')
    
    for i in temp_dataset:
        for j in range(len(i['Item']) - 1):
            if i['Item'][j] + i['Item'][j + 1] not in eucs_dict:
                eucs_dict[i['Item'][j] + i['Item'][j + 1]] = 0
            eucs_dict[i['Item'][j] + i['Item'][j + 1]] += twu_transaction_dict[i['Tid']]
    
    return temp_dataset, eucs_dict

In [54]:
NEW_DATASET, EUCS_LIST = create_periodic_high_util_itemset(dataset=DATASET, unit_utility=UNIT_UTILITY, epm=EPM)

[('a', 5, 3, 75), ('b', 2, 6, 105), ('d', 2, 12, 120), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 5, 3, 75), ('b', 2, 6, 105), ('d', 2, 12, 120)]
[('a', 1, 3, 75), ('d', 1, 12, 120), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 1, 3, 75), ('d', 1, 12, 120)]
[('a', 1, 3, 75), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('b', 3, 6, 105), ('d', 3, 12, 120), ('c', 2, -3, 123)]
[('c', 2, -3, 123), ('b', 3, 6, 105), ('d', 3, 12, 120)]
[('c', 6, -3, 123)]
[('c', 6, -3, 123)]
[]
[]


In [55]:
for i in NEW_DATASET:
    print(i)

{'Tid': 'T1', 'Item': ['c', 'a', 'b', 'd'], 'Quantity': [1, 5, 2, 2]}
{'Tid': 'T2', 'Item': ['c', 'a', 'd'], 'Quantity': [1, 1, 1]}
{'Tid': 'T3', 'Item': ['c', 'a'], 'Quantity': [1, 1]}
{'Tid': 'T4', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T5', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T6', 'Item': ['c', 'b', 'd'], 'Quantity': [2, 3, 3]}
{'Tid': 'T7', 'Item': ['c'], 'Quantity': [6]}
{'Tid': 'T8', 'Item': [], 'Quantity': []}


In [56]:
print(EUCS_LIST)

{'ca': 69, 'ab': 51, 'bd': 105, 'ad': 15, 'cb': 54}


## Searching stage

In [57]:
def construct(prefix_list: list, x: list, y: list) -> list:
    pre_pointer = 0
    x_pointer = 0
    y_pointer = 0
    result = list()

    while x_pointer < len(x) and y_pointer < len(y):
        if x[x_pointer][0] < y[y_pointer][0]:
            x_pointer += 1
        elif x[x_pointer][0] > y[y_pointer][0]:
            y_pointer += 1
        else:
            if prefix_list != None:
                while prefix_list[pre_pointer][0] < x[x_pointer][0]:
                    pre_pointer += 1
                result.append([pre_pointer, x[x_pointer][1] + y[y_pointer][1] - prefix_list[pre_pointer][1], min(x[x_pointer][2], y[y_pointer][2])])
                x_pointer += 1 
                y_pointer += 1
                pre_pointer += 1
            else:
                result.append([x_pointer, x[x_pointer][1] + y[y_pointer][1], min(x[x_pointer][2], y[y_pointer][2])])
                x_pointer += 1
                y_pointer += 1
    return result

def utility_condition(x, minutil):
    return sum(i[1] for i in x) >= minutil

def calculate_periodic(x):
    # print(x)
    periodic = list()
    keys = x
    for i in range(len(keys) - 1):
        periodic.append(keys[i+1][0] - keys[i][0])

    return max(periodic), min(periodic), sum(periodic) / len(periodic)

def periodic_condition(x, minper, maxper, minavg, maxavg):
    max_i, min_i, avg_i = calculate_periodic(x)
    return min_i >= minper and max_i <= maxper and avg_i >= minavg and avg_i <= maxavg

def ru_condition(x, minutil):
    return sum(i[2] for i in x) >= minutil

def max_per_condition(x, maxper):
    max_x, min_x, avg_x = calculate_periodic(x)
    return max_x <= maxper

def avg_per_condition(x, maxavg):
    max_x, min_x, avg_x = calculate_periodic(x)
    return avg_x <= maxavg

def get_name(X, Y, eucs_list):
    return X + Y if X + Y in eucs_list else Y + X

def eucp_condition(X, Y, eucs_list, minutil):
    items_name = get_name(X, Y, eucs_list)
    return eucs_list[items_name] >= minutil



In [58]:
def phm_seach(prefix : str, prefix_list: list, lists: dict) -> set:
    result = set()
    key_lists = list(lists.keys())
    # print(key_lists)
    for i in key_lists:
        print(i)
        if utility_condition(lists[i], EPM['min_util']) and periodic_condition(lists[i], EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg']):
            result.add(i)
        print(lists[i])
        print(EPM)
        if ru_condition(lists[i], EPM['min_util']) and max_per_condition(lists[i], EPM['max_per']) and avg_per_condition(lists[i], EPM['max_avg']):
            new_list = dict()

            for j in key_lists[key_lists.index(i) + 1:]:
                if eucp_condition(i[-1],j[-1], EUCS_LIST, EPM['min_util']):

                    z = get_name(i[-1], j[-1], EUCS_LIST) if prefix == None else prefix + get_name(i[-1], j[-1], EUCS_LIST)
                    
                    z_list = construct(prefix_list, lists[i], lists[j])
                    
                    new_list[z] = z_list
            result.update(phm_seach(i, lists[i], new_list))
            
    return result

In [59]:
print(UNIT_UTILITY)

for i in NEW_DATASET:
    print(i)

{'a': 3, 'b': 6, 'c': -3, 'd': 12, 'e': -5, 'f': -2, 'g': -1}
{'Tid': 'T1', 'Item': ['c', 'a', 'b', 'd'], 'Quantity': [1, 5, 2, 2]}
{'Tid': 'T2', 'Item': ['c', 'a', 'd'], 'Quantity': [1, 1, 1]}
{'Tid': 'T3', 'Item': ['c', 'a'], 'Quantity': [1, 1]}
{'Tid': 'T4', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T5', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T6', 'Item': ['c', 'b', 'd'], 'Quantity': [2, 3, 3]}
{'Tid': 'T7', 'Item': ['c'], 'Quantity': [6]}
{'Tid': 'T8', 'Item': [], 'Quantity': []}


In [60]:
utility_list = dict()
for i in NEW_DATASET:
    total = 0
    for j in range(len(i['Item']) - 1, -1, -1):
        
        if i['Item'][j] not in utility_list:
            utility_list[i['Item'][j]] = list()

        utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])

        total += UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j] if UNIT_UTILITY[i['Item'][j]] > 0 else 0
for i in utility_list.keys():
    print(i, utility_list[i])

d [[1, 24, 0], [2, 12, 0], [6, 36, 0]]
b [[1, 12, 24], [6, 18, 36]]
a [[1, 15, 36], [2, 3, 12], [3, 3, 0], [4, 3, 0], [5, 3, 0]]
c [[1, -3, 51], [2, -3, 15], [3, -3, 3], [6, -6, 54], [7, -18, 0]]


In [61]:
result = phm_seach(None, None, utility_list)
print(result)

d
[[1, 24, 0], [2, 12, 0], [6, 36, 0]]
{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}
b
[[1, 12, 24], [6, 18, 36]]
{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}
a
[[1, 15, 36], [2, 3, 12], [3, 3, 0], [4, 3, 0], [5, 3, 0]]
{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}
ca
[[0, 12, 36], [1, 0, 12], [2, 0, 0]]
{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}
c
[[1, -3, 51], [2, -3, 15], [3, -3, 3], [6, -6, 54], [7, -18, 0]]
{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}
{'d'}


In [62]:
def search(prefix : str, prefix_list: list, lists: list, item_utility: list) -> set:
    result = set()
    for i in lists:
        if utility_condition(i) and periodic_condition(i):
            result.add(i)
        elif ru_condition(i) and max_per_condition(i) and avg_per_condition(i):
            new_list = list()

            for j in lists[lists.index(i) + 1:]:
                if eucp_conditon(i,j):
                    compute_du(j,i)
                    z = i + j # wrong fix it
                    if isinstance(j, m_list):
                        z_list = construct(,i,)


SyntaxError: invalid syntax (2029214967.py, line 14)